In [ ]:
# import juliacall as jl

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [ ]:
# pkgs = "CSV DataFrames ProgressMeter SuiteSparseGraphBLAS KrylovKit Combinatorics Colors JLD2 LaTeXStrings".split()

# for pkg in pkgs:
#     jl.Main.Pkg.add(pkg)

   Resolving package versions...
  No Changes to `~/acband-laughlin-instability/.venv/julia_env/Project.toml`
  No Changes to `~/acband-laughlin-instability/.venv/julia_env/Manifest.toml`
Precompiling project...
┌ Warning: attempting to remove probably stale pidfile
│   path = "/home/cwpark/.julia/compiled/v1.11/SuiteSparseGraphBLAS/HtDaW_d2SyO.ji.pidfile"
└ @ FileWatching.Pidfile ~/acband-laughlin-instability/.venv/julia_env/pyjuliapkg/install/share/julia/stdlib/v1.11/FileWatching/src/pidfile.jl:249
  38022.2 ms  ✓ SuiteSparseGraphBLAS
  1 dependency successfully precompiled in 38 seconds. 100 already precompiled.
   Resolving package versions...
  No Changes to `~/acband-laughlin-instability/.venv/julia_env/Project.toml`
  No Changes to `~/acband-laughlin-instability/.venv/julia_env/Manifest.toml`
   Resolving package versions...
  No Changes to `~/acband-laughlin-instability/.venv/julia_env/Project.toml`
  No Changes to `~/acband-laughlin-instability/.venv/julia_env/Manifest.toml`
 

In [ ]:
# jl.Main.include("julia_ref/code_00_Lattice.jl")
# jl.Main.include("julia_ref/code_01_FiniteLattice.jl")
# jl.Main.include("julia_ref/code_10_InteractionMatrix.jl")
# jl.Main.include("julia_ref/code_20_ManyBodyHamiltonian.jl")

ManyBodyHamiltonian_MK_2V1B_2B_col (generic function with 1 method)

In [ ]:
# fl = jl.Main.TriangularLattice()

In [ ]:
import numpy as np
from typing import Any, Dict, List

In [ ]:
def calculate_wg_fft_py(fl: Dict[str, Any], Qs: List[List[float]], K_param: float, grid_res: int = 128) -> np.ndarray:
    """
    Julia의 calculate_wg_FFT 함수를 Python(Numpy)으로 변환한 버전입니다.

    Args:
        fl: 'lattice' 키를 포함하는 딕셔너리.
            fl['lattice']는 'Q1', 'Q2', 'a1', 'a2' 키를 가져야 합니다.
        Qs: g 벡터들의 리스트 (예: [[g1x, g1y], [g2x, g2y], ...])
        K_param: K 파라미터 (float)
        grid_res: 실공간 그리드 해상도

    Returns:
        np.ndarray: 계산된 w_g 값 (복소수 배열)
    """
    
    # 1. 딕셔너리에서 격자 벡터 가져오기
    # Julia의 fl.lattice.Q1 -> Python의 fl['lattice']['Q1']
    try:
        lattice_data = fl['lattice']
        Q1 = np.array(lattice_data['Q1'])
        Q2 = np.array(lattice_data['Q2'])
        a1 = np.array(lattice_data['a1'])
        a2 = np.array(lattice_data['a2'])
    except KeyError as e:
        raise KeyError(f"fl 딕셔너리에 필요한 키가 없습니다: {e}. 'fl'은 {{'lattice': {{'Q1': ..., 'Q2': ..., 'a1': ..., 'a2': ...}}}} 구조여야 합니다.")
    
    Q3 = -Q1 - Q2

    # 2. 실공간(UC) 그리드 생성
    # u, v: [0, 1) 범위를 grid_res 단계로 나눔
    u = np.arange(grid_res) / grid_res
    v = np.arange(grid_res) / grid_res
    
    # 2D 그리드 생성. r(u,v) = u*a1 + v*a2
    # Julia의 [ ... for i, j] comprehension -> np.meshgrid
    # 'ij' 인덱싱은 Julia의 루프 순서(i가 첫 번째)와 일치합니다.
    uu, vv = np.meshgrid(u, v, indexing='ij')
    
    # r_grid shape: (grid_res, grid_res, D) (D는 벡터 차원)
    # NumPy 브로드캐스팅 활용: (grid_res, grid_res, 1) * (D,) -> (grid_res, grid_res, D)
    r_grid = uu[..., np.newaxis] * a1 + vv[..., np.newaxis] * a2

    # 3. K(r) 및 fvals = |B(r)|^2 계산 (벡터화 연산)
    pref = -np.sqrt(3) * K_param / (4 * np.pi)
    
    # np.tensordot으로 r_grid와 Q 벡터들의 내적을 한 번에 계산
    # (grid_res, grid_res, D)와 (D,)를 내적 -> (grid_res, grid_res)
    dot_Q1_r = np.tensordot(r_grid, Q1, axes=([-1], [0]))
    dot_Q2_r = np.tensordot(r_grid, Q2, axes=([-1], [0]))
    dot_Q3_r = np.tensordot(r_grid, Q3, axes=([-1], [0]))

    K_r = pref * (np.cos(dot_Q1_r) + np.cos(dot_Q2_r) + np.cos(dot_Q3_r))
    fvals = np.exp(-2 * K_r) # fvals shape: (grid_res, grid_res)

    # 4. w_g 계산
    gNum = len(Qs)
    w_g = np.zeros(gNum, dtype=complex)
    
    # Qs 리스트를 numpy 배열로 변환하여 효율성 증대
    Qs_np = np.array(Qs)

    for ig in range(gNum): # 0-based indexing
        g = Qs_np[ig] # 현재 g 벡터
        
        # g와 모든 r 벡터의 내적 계산
        # (grid_res, grid_res, D)와 (D,) 내적 -> (grid_res, grid_res)
        dot_g_r = np.tensordot(r_grid, g, axes=([-1], [0]))
        
        # 위상항(phasor) 계산 (Julia의 1im -> Python의 1j)
        phasor = np.exp(1j * dot_g_r)
        
        # 합계: s = Σ_r fvals[r] * exp(i * g·r)
        # NumPy의 sum을 사용하여 중첩 루프 대체
        s = np.sum(fvals * phasor)
        
        w_g[ig] = s / (grid_res**2)

    return w_g